In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/dataanalysis/data

Mounted at /content/drive
/content/drive/MyDrive/dataanalysis/data


In [2]:

%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium, 이거는 내부에서 실행을 해야한다.
pip install selenium


Executing: /tmp/apt-key-gpghome.Mo5mZqyRL9/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.vbdOU9ejLk/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.pNQ6moxP21/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [3]:
# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

# 시간 대기 작업할 때 필요함
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
# Colab에선 웹브라우저 창이 뜨지 않음으로 별도 설정한다

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [57]:
import openpyxl

# 엑셀 파일 생성
wb = openpyxl.Workbook()

# Sheet 워크시트 삭제
wb.remove_sheet(wb['Sheet'])


#해당 url로 이동
url = "https://naver.com" 
driver.get(url)

stock = driver.find_element(By.CSS_SELECTOR,'#NM_FAVORITE > div.group_nav > ul.list_nav.NM_FAVORITE_LIST > li:nth-child(3) > a')
stock.click()

rank_btn = driver.find_element(By.CSS_SELECTOR,'div.article > div.section > div.section_sise_top > ul > li.tab1 > a')
rank_btn.click()
# time.sleep(5) # 5초 동안 대기 하라


for i in range(1,5) :
  s_list = driver.find_element(By.CSS_SELECTOR,'#content > div.article > div.section > div.section_sise_top > ul > li.tab'+str(i)+' > a')
  s_list.click()

  #워크 시트 생성
  # 워크시트 생성(index=1을 설정하면 가장 최근 워크시트가 제일 앞으로 배치됨)
  ws = wb.create_sheet(s_list.text)   
  ws['A1'] = '종목'  
  ws['B1'] = '현재가'  
  ws['C1'] = '전일대비'
  ws['D1'] = '등락률'
  # print(s_list.text)


  div_stock_list = driver.find_elements(By.CSS_SELECTOR, "#_topItems" + str(i) + ">tr")
  for idx, element in enumerate(div_stock_list, start=1):
   stock_name = element.find_element(By.CSS_SELECTOR,'#_topItems'+str(i)+'> tr:nth-child('+str(idx)+') > th > a' ) # 종목
   cur_price = element.find_element(By.CSS_SELECTOR,'#_topItems'+str(i)+' > tr:nth-child('+str(idx)+') > td:nth-child(2)') # 현재가
   diff = element.find_element(By.CSS_SELECTOR,'#_topItems'+str(i)+' > tr:nth-child('+str(idx)+') > td:nth-child(3)') # 전일대비
   percen = element.find_element(By.CSS_SELECTOR,'#_topItems'+str(i)+' > tr:nth-child('+str(idx)+') > td:nth-child(4)') # 등락율
                                

   print(stock_name.text, cur_price.text,diff.text,percen.text)


   ws['A'+str(idx+1)] = stock_name.text
   ws['B'+ str(idx+1)] = cur_price.text
   ws['C'+ str(idx+1)] = diff.text
   ws['D'+ str(idx+1)] = percen.text


wb.save('./stock.xlsx')

<ipython-input-57-7a2af0827aef>:7: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  wb.remove_sheet(wb['Sheet'])


KODEX 200선물인버스2X 2,845 상승
50 +1.79%
미래아이앤지 473 상승
25 +5.58%
유니온머티리얼 3,175 상승
450 +16.51%
KODEX 코스닥150선물인버스 4,810 상승
110 +2.34%
삼성 인버스 2X WTI원유 선물 ETN 135 상승
5 +3.85%
KODEX 인버스 4,735 상승
45 +0.96%
성우하이텍 6,960 상승
730 +11.72%
셀바스AI 19,630 상승
2,550 +14.93%
KODEX 레버리지 15,310 하락
290 -1.86%
미래나노텍 22,100 상승
2,640 +13.57%
쎄노텍 1,890 상승
69 +3.79%
KODEX 코스닥150레버리지 8,075 하락
330 -3.93%
MDS테크 2,450 상승
35 +1.45%
키이스트 11,590 상승
2,080 +21.87%
에스아이리소스 523 상승
71 +15.71%
유니온 6,280 상한가
1,445 +29.89%
안트로젠 23,000 상한가
5,290 +29.87%
화인써키트 20,250 상한가
4,650 +29.81%
아이에스동서 46,650 상승
9,000 +23.90%
키이스트 11,590 상승
2,080 +21.87%
BYC 472,500 상승
79,500 +20.23%
효성화학 141,800 상승
23,800 +20.17%
린드먼아시아 7,650 상승
1,250 +19.53%
유니온머티리얼 3,175 상승
450 +16.51%
삼영이엔씨 5,010 상승
695 +16.11%
에스아이리소스 523 상승
71 +15.71%
셀바스AI 19,630 상승
2,550 +14.93%
미래나노텍 22,100 상승
2,640 +13.57%
대신밸런스제10호스팩 4,375 상승
510 +13.20%
원림 25,000 상승
2,900 +13.12%
스튜디오산타클로스 754 하락
262 -25.79%
다이나믹디자인 3,575 하락
725 -16.86%
계양전기우 5,000 하락
800 -13.79%
인화정공 13,010 하락
1,64

In [ ]:
# 종료
# driver.quit()